In [ ]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd

CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'

Gtk-Message: 15:30:08.862: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/E-regular-binding-site-predictor/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptoshow-analysis/src/E-regular-binding-site-predictor/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'
 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.
PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.
[{"variableName": "binding_residues", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": false, "rawType": "builtins.dict"}, {"variableName": "binding_residues_mmcifed", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": false, "rawType": "builtins.dict"}, {"variableName": "model_predictions", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": false, "rawType": "builtins.dict"}]
[{"variableName": "binding_residues", "type": "dictionary", "supportedEngines": ["pandas"], "isLocalVariable": false, "rawType": "builtins.dict"}, {"variableName": "binding_residues_mmcifed", "type": "dictionary",

In [16]:
import sys
import numpy as np

DATA_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis_for_pocket_level_evaluation.csv'
PREDICTIONS_PATH = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/predictions'
PRECOMPUTED = True

sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/D-visualize')
import vis_utils

sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/B-evaluate-cryptoshow')
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import eval_utils
import cryptoshow_utils

# load ground truth binding residues: these have mmcif numbering and need to be mapped to auth labeling
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)

# load predictions
model_predictions_mmcifed = vis_utils.read_predictions(data_path=f'{PREDICTIONS_PATH}/finetuning-with-smoothing', protein_ids=binding_residues_mmcifed.keys())

if not PRECOMPUTED:
    # map binding residues to auth labeling
    binding_residues = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket, auth=False) for pocket in pockets] for protein_id, pockets in binding_residues_mmcifed.items()}
    model_predictions = {protein_id: [cryptoshow_utils.map_mmcif_numbering_to_auth(protein_id[:4], protein_id[4:], pocket, auth=False) for (pocket, _) in pockets] for protein_id, pockets in model_predictions_mmcifed.items()}

else:
    import pickle
    with open(f'{PREDICTIONS_PATH}/model_predictions.pkl', 'rb') as f:
        model_predictions = pickle.load(f)
    with open(f'{PREDICTIONS_PATH}/binding_residues.pkl', 'rb') as f:
        binding_residues = pickle.load(f)


In [10]:
len(binding_residues['1a52A'])

3

In [13]:
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src/B-evaluate-cryptoshow')
sys.path.append('/home/vit/Projects/cryptoshow-analysis/src')
import eval_utils
binding_residues_mmcifed, _ = eval_utils.read_test_binding_residues(data_path=DATA_PATH, pocket_types=['NON_CRYPTIC'])
binding_residues_mmcifed = vis_utils.reformat_binding_residues(binding_residues_mmcifed)
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'
np.load(f'{COORDINATES_DIR}/1a52A.npy').shape

(239, 3)

In [11]:
# import pickle
# 
# with open(f'{PREDICTIONS_PATH}/model_predictions.pkl', 'wb') as f:
#     pickle.dump(model_predictions, f)
# with open(f'{PREDICTIONS_PATH}/binding_residues.pkl', 'wb') as f:
#     pickle.dump(binding_residues, f)
np.load(f'{COORDINATES_DIR}/1a52A.npy')

NameError: name 'COORDINATES_DIR' is not defined

In [ ]:
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/E-regular-binding-site-predictor/ligysis-coordinates'

for protein_id in binding_residues.keys():
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.set('cif_use_auth', 0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    
    if len(this_model_predictions) != 0:
        cmd.color('forest', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
        intersection1 = vis_utils.get_intersection(this_binding_residues, this_model_predictions)
        if len(intersection1) != 0:
            cmd.color('yellow', vis_utils.generate_pymol_algebra_selection(protein_id, intersection1))


    DCCs = vis_utils.compute_DCCs(protein_id, [i for (i, _) in model_predictions_mmcifed[protein_id]], binding_residues_mmcifed[protein_id], COORDINATES_DIR)
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')
    
    vis_utils.draw_pocket_centers(binding_residues_mmcifed[protein_id], [i for (i, _) in model_predictions_mmcifed[protein_id]], protein_id, COORDINATES_DIR)
    vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], [i for (i, _) in model_predictions_mmcifed[protein_id]], protein_id)

    # cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

In [19]:
binding_residues_mmcifed[protein_id]

[array([ 44, 111, 224,  45, 113, 117, 217,  33,  47, 119,  98, 109, 114,
        215,  38, 218, 118, 110, 214, 223, 225, 102, 104, 122, 219, 230,
        233, 234, 235, 238,  39,  40,  43,  48,  78,  81,  85,  96, 115,
        211, 216, 222, 237,  36,  37,  82, 226, 229, 108,  88, 212, 221,
         35,  41, 112, 116, 227, 228,  77, 220, 231]),
 array([111,  26,  24,  31, 102, 103, 104,  39,  36, 101,  27,  30, 108,
         29,  23,  32,  35,  25]),
 array([24, 20, 21, 46, 22])]

In [15]:
binding_residues[protein_id]

[[43,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  53,
  54,
  55,
  57,
  58,
  87,
  88,
  91,
  92,
  95,
  98,
  106,
  108,
  112,
  114,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  132,
  221,
  222,
  224,
  225,
  226,
  227,
  228,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  238,
  239,
  240,
  241,
  243,
  244,
  245,
  247,
  248],
 [33,
  34,
  35,
  36,
  37,
  39,
  40,
  41,
  42,
  45,
  46,
  49,
  111,
  112,
  113,
  114,
  118,
  121],
 [30, 31, 32, 34, 56]]

In [8]:
this_binding_residues

array([ 43,  45,  46,  47,  48,  49,  50,  51,  53,  54,  55,  57,  58,
        87,  88,  91,  92,  95,  98, 106, 108, 112, 114, 118, 119, 120,
       121, 122, 123, 124, 125, 126, 127, 128, 129, 132, 221, 222, 224,
       225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
       238, 239, 240, 241, 243, 244, 245, 247, 248,  33,  34,  35,  36,
        37,  39,  40,  41,  42,  45,  46,  49, 111, 112, 113, 114, 118,
       121,  30,  31,  32,  34,  56])

In [8]:
COORDINATES_DIR = '/home/vit/Projects/cryptoshow-analysis/data/A-cluster-ligysis-data/coordinates'
def draw_pocket_centers(binding_residues, predicted_residues, protein_id, color='blue'):
    from pymol import cmd
    from pymol.cgo import COLOR, SPHERE
    coordinates = np.load(f'{COORDINATES_DIR}/{protein_id.replace("_", "")}.npy')
    for i, pocket in enumerate(binding_residues):
        center = coordinates[pocket].mean(axis=0)
        spherelist = [
           COLOR,    1.000,    0.000,    0.000,
           SPHERE,   center[0],   center[1],   center[2], 0.30,
            ]
        cmd.load_cgo(spherelist, f'true_pocket_{i}', 1)
        cmd.pseudoatom(f'true_pocket_center_{i}', pos=center.tolist())

    for i, pocket in enumerate(predicted_residues):
        center = coordinates[pocket].mean(axis=0)
        if color == 'green':
            spherelist = [
               COLOR,    0.000,    1.000,    0.000,
               SPHERE,   center[0],   center[1],   center[2], 0.30,
                ]
        else:
            spherelist = [
               COLOR,    0.000,    0.000,    1.000,
               SPHERE,   center[0],   center[1],   center[2], 0.30,
                ]
        cmd.load_cgo(spherelist, f'predicted_pocket_{i}', 1)
        cmd.pseudoatom(f'predicted_pocket_center_{i}_{color}', pos=center.tolist())
        for j, true_pocket in enumerate(binding_residues):
            cmd.distance(f'distance_{i}_{j}', f'true_pocket_center_{j}', f'predicted_pocket_center_{i}_{color}')

for protein_id in binding_residues.keys():
    this_p2rank_predictions = np.concatenate(p2rank_predictions[protein_id]) if p2rank_predictions[protein_id] else np.array([])
    this_model_predictions = np.concatenate(model_predictions[protein_id]) if model_predictions[protein_id] else np.array([])
    this_binding_residues = np.concatenate(binding_residues[protein_id]) if binding_residues[protein_id] else np.array([])
    
    merged_predictions = p2rank_predictions_mmcifed[protein_id] + model_predictions_mmcifed[protein_id]
    DCCs = vis_utils.compute_DCCs(protein_id, binding_residues_mmcifed[protein_id], merged_predictions)
    successful_predictions = vis_utils.count_successful_predictions(DCCs)
    print(f'{successful_predictions} / {len(binding_residues_mmcifed[protein_id])} pockets found. {DCCs}')

    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)
    if len(this_binding_residues) != 0:
        cmd.color('red', vis_utils.generate_pymol_algebra_selection(protein_id, this_binding_residues))
    else:
        continue
    
    if len(this_model_predictions) != 0:
        cmd.color('forest', vis_utils.generate_pymol_algebra_selection(protein_id, this_model_predictions))
        intersection1 = vis_utils.get_intersection(this_binding_residues, this_model_predictions)
        if len(intersection1) != 0:
            cmd.color('lime', vis_utils.generate_pymol_algebra_selection(protein_id, intersection1))


    if len(this_p2rank_predictions) != 0:
        cmd.color('blue', vis_utils.generate_pymol_algebra_selection(protein_id, this_p2rank_predictions))
        intersection2 = vis_utils.get_intersection(this_binding_residues, this_p2rank_predictions)
        if len(intersection2) != 0:
            cmd.color('cyan', vis_utils.generate_pymol_algebra_selection(protein_id, intersection2))

        intersection_of_all = vis_utils.get_intersection(intersection1, intersection2)
        if len(intersection_of_all) != 0:
            cmd.color('yellow', vis_utils.generate_pymol_algebra_selection(protein_id, intersection_of_all))
    
    draw_pocket_centers(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)
    draw_pocket_centers(binding_residues_mmcifed[protein_id], model_predictions_mmcifed[protein_id], protein_id, color='green')
    # vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], p2rank_predictions_mmcifed[protein_id], protein_id)
    # vis_utils.draw_pocket_center_lines(binding_residues_mmcifed[protein_id], model_predictions_mmcifed[protein_id], protein_id)

    # cmd.show('surface', protein_id)
    cmd.zoom(protein_id)
    user_input = input(">Press Enter for the next protein (press 'q' to quit)...\n")
    if user_input.lower() == 'q':
        break

20.998554
19.656574
21.088614
14.021774
41.05787
21.980396
22.402721
18.067108
20.223907
22.745678
0 / 1 pockets found. [14.021774]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
7.8622446
3.473199
15.464708
18.02419
23.794159
15.6300745
12.774979
5.4894896
1 / 1 pockets found. [3.473199]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
10.244024
16.52279
14.475388
14.372983
16.831547
0 / 1 pockets found. [10.244024]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
PyMOL>show surface
4.0950685
21.793299
18.812605
8.967337
11.430943
0 / 1 pockets found. [4.0950685]
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
